In [1]:
import os
import requests
import datetime

import numpy as np
import pandas as pd

from bs4 import BeautifulSoup

In [2]:
def pandas_settings() -> None:
        for option in ('display.max_rows', 'display.max_columns', 'display.width', 'display.memory_usage'):
            pd.set_option(option, 250 if 'memory_usage' not in option else False)
        # message('pandas')
        return None
pandas_settings()

In [3]:
class Conversions:
    
    def __init__(self):
        
        
        
        self.inits_teams: dict[str, str] = {
            'NY': 'New York Knicks',
            'LAL': 'Los Angeles Lakers',
            'MIA': 'Miami Heat',
            'UTA': 'Utah Jazz',
            'PHO': 'Phoenix Suns',
            'LAC': 'Los Angeles Clippers',
            # 'LAC': 'LA Clippers',
            'PHI': 'Philadelphia 76ers',
            'DAL': 'Dallas Mavericks',
            'DEN': 'Denver Nuggets',
            'BOS': 'Boston Celtics',
            'ATL': 'Atlanta Hawks',
            'CLE': 'Cleveland Cavaliers',
            'DET': 'Detroit Pistons',
            'TOR': 'Toronto Raptors',
            'CHA': 'Charlotte Hornets',
            'ORL': 'Orlando Magic',
            'MEM': 'Memphis Grizzlies',
            'SA': 'San Antonio Spurs',
            'MIL': 'Milwaukee Bucks',
            'IND': 'Indiana Pacers',
            'CHI': 'Chicago Bulls',
            'OKC': 'Oklahoma City Thunder',
            'GS': 'Golden State Warriors',
            'HOU': 'Houston Rockets',
            'BKN': 'Brooklyn Nets',
            'POR': 'Portland Trail Blazers',
            'NO': 'New Orleans Pelicans',
            'MIN': 'Minnesota Timberwolves',
            'SAC': 'Sacramento Kings',
            'WAS': 'Washington Wizards'
        }

        # Invert
        self.teams_inits: dict[str,str] = { val: key for key, val in self.inits_teams.items() }
        
    def team_name(self, team_str: str) -> str:
        return self.teams_inits[team_str]
    
    def team_initials(self, team_init_str: str) -> str:
        return self.inits_teams[team_init_str]

In [4]:
class PropsScraper:
    
    def __init__(self):
        self.convert = Conversions()
        self.directory_url: str = 'https://www.scoresandodds.com/nba/players'
        
        self.current_date_str = datetime.datetime.now().strftime("%m/%d")
    
#     Creates a dictionary containing the links to current and historical props
#     for every player in the NBA, organized by team
    def create_webpage_directory(self) -> dict[str, dict[str, str]]:
        
#         Load HTML into bs4
        soup = BeautifulSoup(
            requests.get(self.directory_url).text,
            'html.parser'
        )

#         Load each team data into dictionary, converting the full team name into initials as used in rest of data
        team_modules = {
            self.convert.team_name(team_html.find('h3').get_text()): team_html.find_all('div', class_='module-body')[0].find('ul')
            for team_html in soup.find_all('div', class_='module')
        }
        
        
#         Parse HTML data for each team to organize links in easily searchable manner
        teams_players_links: dict[str, dict[str, str]] = {
            
            team: {
                # a_tag.get_text(): self.directory_url.replace(
                #     '/nba/players',
                #     a_tag['href']
                # )
                ' '.join(a_tag.get_text().split(' ')[:2]): self.directory_url.replace(
                    '/nba/players',
                    a_tag['href']
                )
                for a_tag in module.find_all('a')
            }
            
            for team, module in team_modules.items()
            
        }
        
        return teams_players_links
    
    # Implied Probability = 100 / (Odds + 100)
    @staticmethod
    def pos_ml_prob(ml: str) -> float:
        return 100 / sum([int(ml[1:]),100])
    
    # Implied Probability = (-1*(Odds)) / (-1(Odds) + 100) ->
    @staticmethod
    def neg_ml_prob(ml: str) -> float:
        ml: int = int(ml)
        return (-1*ml) / sum([-1*ml,100])
        
    @classmethod
    def implied_probability(cls, ml: str):
        if ml == '+100':
            return 0.5
        
        return cls.pos_ml_prob(ml) if ml[0]=='+' else cls.neg_ml_prob(ml)
    
    @classmethod
    def expected_value(cls, val: float, ml: str) -> float:
        return cls.implied_probability(ml)*val
        
    def scrape_player_props(self, name, url):
        
#         Load HTML
        soup = BeautifulSoup(
            requests.get(url).text, 
            'html.parser'
        )
        
        module = soup.find('div', class_="module-body scroll")
        
        if not len(module.find_all('span')):
            return (0.0,0.0)
        
#         Make sure current
        zerofill = lambda dp: f'0{dp}' if len(dp) == 1 else dp
        date_str = '/'.join([
            zerofill(dp) for dp in module.find_all('span')[2].get_text().split(' ')[1].split('/')
        ])
        
        if date_str != self.current_date_str:
            return (0.0,0.0)
        
        props_rows = module.find('table', class_='sticky').find('tbody').find_all('tr')
        
        # Steals, blocks, 3s are options but noisy, better to use season data for opponents
        targets = ('Points', 'Rebounds', 'Assists')
        
#         Form: Category Line Over Under
        target_rows = [row for row in props_rows if row.find('td').get_text() in targets]
    
    
#     TODO: Figure out more efficient way for this, dict(zip()) probably best
        props = {
            'stat': list(),
            'value': list(),
            # 'over': list(),
            'e_value': list(),
            'fpts': list(),
            'e_fpts': list()
            # 'under': list()
        }
        
        multipliers: dict[str,float] = {'assists': 1.5, 'rebounds': 1.2}
        for rowtags in target_rows:
            vals = [val.get_text().lower() for val in rowtags.find_all('td')] # (Category, Line, Over, Under)
            
            stat: str = vals[0]
            props['stat'].append(stat)
            
            statval = sum([float(vals[1]), 0.5])
            props['value'].append(statval)
            
            overml: str = vals[2]
            
            props['e_value'].append(self.expected_value(statval, overml))
            
            multi: float = multipliers.get(stat, 1.0)
            fpts: float = multi*statval
            
            props['fpts'].append(fpts)
            props['e_fpts'].append(self.expected_value(fpts, overml))
            # props['under'].append(vals[3])
        
        
        df: pd.DataFrame = pd.DataFrame(props).round(2)
        
        
        return (df['fpts'].sum(), df['e_fpts'].sum())

In [5]:
# Incomplete rn
Props = PropsScraper()
directory: dict[str,dict[str,str]] = Props.create_webpage_directory()

def test_(name: str, team: str, **kwargs):
    fp, efp = Props.scrape_player_props(
        name,
        directory[team][name]
    )
    
    return (fp, efp) #if kwargs.get('salary') is None else tuple([ 1000 * (fp_/kwargs['salary']) for fp_ in (fp,efp) ])

In [24]:
# Temporary, just figuring out dynamics for now

def load_slate():
    
    path: str = '../data/current-fanduel.csv'
    
    columns: dict[str, str] = {
        'Nickname': 'name',
        'Position': 'pos',
        'Team': 'team',
        'Salary': 'salary'
    }
    
    df: pd.DataFrame = (pd
                        .read_csv(path, usecols=columns)
                        .rename(columns,axis=1)
                        .pipe(lambda df_: df_.loc[df_['salary']>=4000])
                       )
    
    df['name'] = df['name'].str.replace('.','', regex=False)
    
    issues = {'Luguentz Dort': 'Lu Dort', 'Moritz Wagner':'Moe Wagner'}
    df['name'] = df['name'].map(lambda x: issues.get(x,x))
    
    df['input'] = tuple(zip(df['name'], df['team']))
    df['output'] = df['input'].apply(lambda x: test_(x[0],x[1]))
    
    df['fpts'] = df['output'].map(lambda x: x[0])
    df['e_fpts'] = df['output'].map(lambda x: x[1])
    
    
    for col in ('fpts', 'e_fpts'):
        df[f'{col}/$'] = 1000 * (df[col] / df['salary'])
    
    df['5x'] = 5 * (df['salary'] / 1000)
    df['value'] = df['fpts'] - df['5x']
    
    df = (df
          .loc[df['fpts']>0.0]
          .drop(['input', 'output', '5x'], axis=1)
          .sort_values('value', ascending=False)
          .set_index('name')
          .round(2)
         )
    
    df.to_csv('../fanduel-props.csv')
    
    return df

In [25]:
load_slate()

,pos,salary,team,fpts,e_fpts,fpts/$,e_fpts/$,value
name,,,,,,,,
Brandon Ingram,SF,8300,NO,44.4,23.40,5.35,2.82,2.9
Lauri Markkanen,PF/SF,8600,UTA,45.3,23.74,5.27,2.76,2.3
Damian Lillard,PG,10800,POR,56.2,28.98,5.20,2.68,2.2
Terry Rozier,SG/PG,7800,CHA,40.5,20.95,5.19,2.69,1.5
Josh Giddey,SG/PG,8000,OKC,40.6,22.28,5.08,2.78,0.6
Kelly Oubre,SF/SG,7000,CHA,35.4,17.85,5.06,2.55,0.4
Kris Dunn,PG,4500,UTA,22.3,12.41,4.96,2.76,-0.2
Talen Horton-Tucker,SG/SF,6300,UTA,31.0,17.59,4.92,2.79,-0.5
Jordan Poole,PG/SG,7600,GS,37.3,20.22,4.91,2.66,-0.7


In [30]:
(pd
 .read_csv('../fanduel-props.csv')
 .set_index('name')
 # .sort_values('e_fpts/$', ascending=False)
 # .sort_values(['pos', 'e_fpts/$'], ascending=[True, False])
 .sort_values([ 'salary', 'e_fpts/$'], ascending=[True, False])
)

,pos,salary,team,fpts,e_fpts,fpts/$,e_fpts/$,value
name,,,,,,,,
Tre Mann,PG,4000,OKC,9.0,5.09,2.25,1.27,-11.0
Lonnie Walker,SG/SF,4000,LAL,9.0,5.09,2.25,1.27,-11.0
Dario Saric,PF/C,4100,OKC,18.0,9.93,4.39,2.42,-2.5
Anthony Lamb,SF,4100,GS,14.8,8.00,3.61,1.95,-5.7
Trey Lyles,PF,4200,SAC,14.3,8.23,3.40,1.96,-6.7
Reggie Jackson,PG,4200,DEN,14.9,7.95,3.55,1.89,-6.1
Shaedon Sharpe,SG,4300,POR,14.1,8.34,3.28,1.94,-7.4
Rudy Gay,PF/SF,4300,UTA,15.8,7.89,3.67,1.83,-5.7
Bol Bol,C,4300,ORL,13.5,6.98,3.14,1.62,-8.0
